# AiiDA Integration Strategies for Python Packages

This notebook explores how to design a Python package that can work both with and without AiiDA, addressing profile management, optional integration, and entry point strategies.

## Key Questions Addressed:
1. How to handle AiiDA profile requirements gracefully
2. Should CLI code always load profiles?
3. Where to place AiiDA-dependent classes (core vs cli)?
4. Entry points: CLI vs direct Python integration
5. Best practices for optional AiiDA support

---

## 1. AiiDA Profile Error in Jupyter

Let's first reproduce the error you encountered to understand when it occurs:

In [1]:
# Demonstrate the AiiDA profile error
import sys

print("Attempting to import AiiDA...")
try:
    from aiida.orm import ArrayData
    print("✓ AiiDA import successful")
    
    print("Attempting to create ArrayData node...")
    node = ArrayData()
    print("✓ ArrayData created successfully")
    
except ImportError as e:
    print(f"❌ AiiDA not available: {e}")
except Exception as e:
    print(f"❌ AiiDA error: {e}")
    print(f"Error type: {type(e).__name__}")
    
    # This is the ConfigurationError you encountered
    if "profile" in str(e).lower():
        print("\n💡 This is the profile configuration error!")
        print("   AiiDA requires a configured profile before creating nodes.")

Attempting to import AiiDA...


✓ AiiDA import successful
Attempting to create ArrayData node...
❌ AiiDA error: Could not determine the current profile. Consider loading a profile using `aiida.load_profile()`.
Error type: ConfigurationError

💡 This is the profile configuration error!
   AiiDA requires a configured profile before creating nodes.


## 2. Checking for AiiDA Profile in Python Code

Here's how to programmatically check if an AiiDA profile is loaded and load one if needed:

In [2]:
def check_aiida_profile():
    """Check if AiiDA profile is loaded and attempt to load if needed."""
    try:
        import aiida
        from aiida.manage import get_manager
        
        # Check if a profile is already loaded
        try:
            manager = get_manager()
            profile = manager.get_profile()
            print(f"✓ AiiDA profile loaded: {profile.name}")
            return True, profile.name
        except Exception as e:
            print(f"⚠ No AiiDA profile loaded: {e}")
            
            # Try to load a profile automatically
            try:
                aiida.load_profile()
                manager = get_manager()
                profile = manager.get_profile()
                print(f"✓ AiiDA profile auto-loaded: {profile.name}")
                return True, profile.name
            except Exception as load_error:
                print(f"❌ Failed to load AiiDA profile: {load_error}")
                return False, None
                
    except ImportError:
        print("❌ AiiDA not installed")
        return False, None

# Test the profile checker
profile_loaded, profile_name = check_aiida_profile()
print(f"\nProfile status: {'Loaded' if profile_loaded else 'Not loaded'}")
if profile_name:
    print(f"Profile name: {profile_name}")

⚠ No AiiDA profile loaded: 'NoneType' object has no attribute 'name'
✓ AiiDA profile auto-loaded: presto-1

Profile status: Loaded
Profile name: presto-1


## 3. Optional AiiDA Integration: Import and Profile Handling

Here's how to design classes that gracefully handle AiiDA availability and profile status:

In [3]:
# Pattern 1: Safe AiiDA imports with fallback
try:
    import aiida
    from aiida.orm import ArrayData
    from aiida.manage import get_manager
    AIIDA_AVAILABLE = True
    
    # Check for profile on import
    try:
        get_manager().get_profile()
        AIIDA_PROFILE_LOADED = True
    except:
        AIIDA_PROFILE_LOADED = False
        
except ImportError:
    ArrayData = None
    AIIDA_AVAILABLE = False
    AIIDA_PROFILE_LOADED = False

print(f"AiiDA available: {AIIDA_AVAILABLE}")
print(f"AiiDA profile loaded: {AIIDA_PROFILE_LOADED}")

# Pattern 2: Smart factory function with auto-profile loading
def create_aiida_node_safe(data_dict, auto_load_profile=True):
    """Create AiiDA ArrayData node with automatic profile handling."""
    
    if not AIIDA_AVAILABLE:
        raise ImportError("AiiDA not available. Install aiida-core.")
    
    # Try to ensure profile is loaded
    if not AIIDA_PROFILE_LOADED and auto_load_profile:
        try:
            aiida.load_profile()
            print("✓ AiiDA profile auto-loaded")
        except Exception as e:
            raise RuntimeError(f"Cannot load AiiDA profile: {e}")
    
    # Create the node
    try:
        node = ArrayData()
        for key, value in data_dict.items():
            node.set_array(key, value)
        return node
    except Exception as e:
        if "profile" in str(e).lower():
            raise RuntimeError(
                f"AiiDA profile error: {e}\n"
                "Consider calling aiida.load_profile() before using AiiDA nodes."
            )
        raise

# Test the safe factory
import numpy as np

test_data = {
    'time': np.linspace(0, 1, 10),
    'stress': np.random.rand(10)
}

try:
    node = create_aiida_node_safe(test_data)
    print("✓ AiiDA node created successfully")
    print(f"  Arrays: {node.get_arraynames()}")
except Exception as e:
    print(f"❌ Failed to create AiiDA node: {e}")

AiiDA available: True
AiiDA profile loaded: True
✓ AiiDA node created successfully
  Arrays: ['time', 'stress']


## 4. Design Patterns for Optional AiiDA Support

Let's explore different architectural patterns for integrating AiiDA optionally:

In [4]:
# Pattern A: Separate Core and AiiDA modules
# core/response_data.py - No AiiDA dependencies
class ResponseData:
    """Core data class - works without AiiDA"""
    def __init__(self, time, stress, strain):
        self.time = time
        self.stress = stress 
        self.strain = strain
    
    def to_dict(self):
        return {
            'time': self.time.tolist(),
            'stress': self.stress.tolist(), 
            'strain': self.strain.tolist()
        }

# aiida_plugin/response_data_node.py - AiiDA-specific
class ResponseDataNodeMixin:
    """Mixin that adds AiiDA functionality when available"""
    
    @classmethod
    def ensure_aiida_ready(cls):
        """Ensure AiiDA is available and profile loaded"""
        if not AIIDA_AVAILABLE:
            raise ImportError("AiiDA not available")
        
        try:
            get_manager().get_profile()
        except:
            try:
                aiida.load_profile()
            except Exception as e:
                raise RuntimeError(f"Cannot load AiiDA profile: {e}")
    
    def to_aiida_node(self, auto_load_profile=True):
        """Convert to AiiDA ArrayData node"""
        if auto_load_profile:
            self.ensure_aiida_ready()
            
        node = ArrayData()
        for key, value in self.to_dict().items():
            node.set_array(key, np.array(value))
        return node

# Pattern B: Adapter pattern
class AiiDAAdapter:
    """Adapter for AiiDA functionality"""
    
    def __init__(self, auto_setup=True):
        self.available = AIIDA_AVAILABLE
        self.profile_loaded = AIIDA_PROFILE_LOADED
        
        if auto_setup and self.available and not self.profile_loaded:
            try:
                aiida.load_profile()
                self.profile_loaded = True
            except:
                pass
    
    def is_ready(self):
        return self.available and self.profile_loaded
    
    def create_array_node(self, data_dict):
        if not self.is_ready():
            raise RuntimeError("AiiDA not ready")
        
        node = ArrayData()
        for key, value in data_dict.items():
            node.set_array(key, np.array(value))
        return node

# Test the patterns
print("=== Testing Core ResponseData ===")
import numpy as np
rd = ResponseData(
    time=np.linspace(0, 1, 5),
    stress=np.random.rand(5),
    strain=np.random.rand(5)
)
print(f"ResponseData created: {list(rd.to_dict().keys())}")

print("\n=== Testing AiiDA Adapter ===")
adapter = AiiDAAdapter()
print(f"AiiDA adapter ready: {adapter.is_ready()}")

if adapter.is_ready():
    try:
        node = adapter.create_array_node(rd.to_dict())
        print(f"✓ AiiDA node created with arrays: {node.get_arraynames()}")
    except Exception as e:
        print(f"❌ Failed to create node: {e}")
else:
    print("⚠ AiiDA adapter not ready - working in standalone mode")

=== Testing Core ResponseData ===
ResponseData created: ['time', 'stress', 'strain']

=== Testing AiiDA Adapter ===
AiiDA adapter ready: True
✓ AiiDA node created with arrays: ['time', 'stress', 'strain']


## 5. Entry Points: CLI vs Python Integration

You're absolutely correct! Entry points can be defined for direct Python integration without CLI. Let's explore both approaches:

In [5]:
# Entry Points Configuration Examples

# === setup.cfg approach ===
setup_cfg_example = """
[options.entry_points]
# CLI entry points
console_scripts =
    gsm-simulate = bmcs_matmod.gsm_lagrange.cli.cli_gsm:main
    gsm-analyze = bmcs_matmod.gsm_lagrange.cli.analysis:main

# AiiDA calculation entry points (direct Python integration)
aiida.calculations =
    gsm.lagrange = bmcs_matmod.gsm_lagrange.aiida_plugin:GSMLagrangeCalculation

# AiiDA data entry points
aiida.data =
    gsm.response = bmcs_matmod.gsm_lagrange.aiida_plugin.response_data_node:ResponseDataNode

# AiiDA workflow entry points  
aiida.workflows =
    gsm.material_characterization = bmcs_matmod.gsm_lagrange.workflows:MaterialCharacterizationWorkChain

# Custom plugin entry points (for discoverability)
bmcs_matmod.gsm_models =
    elastic_damage = bmcs_matmod.gsm_lagrange.models.gsm1d_ed:GSM1D_ED
    elastic_plastic = bmcs_matmod.gsm_lagrange.models.gsm1d_ep:GSM1D_EP
"""

# === pyproject.toml approach ===
pyproject_toml_example = """
[project.entry-points."console_scripts"]
gsm-simulate = "bmcs_matmod.gsm_lagrange.cli.cli_gsm:main"

[project.entry-points."aiida.calculations"] 
"gsm.lagrange" = "bmcs_matmod.gsm_lagrange.aiida_plugin:GSMLagrangeCalculation"

[project.entry-points."aiida.data"]
"gsm.response" = "bmcs_matmod.gsm_lagrange.aiida_plugin.response_data_node:ResponseDataNode"

[project.entry-points."bmcs_matmod.gsm_models"]
"elastic_damage" = "bmcs_matmod.gsm_lagrange.models.gsm1d_ed:GSM1D_ED"
"""

print("Entry Points provide multiple integration paths:")
print("1. CLI commands (console_scripts)")
print("2. AiiDA calculations (direct Python, no CLI needed)")
print("3. AiiDA data types (for type discovery)")
print("4. Custom plugin discovery")

# === Demonstration: Using entry points programmatically ===
# NOTE: pkg_resources is deprecated, using importlib.metadata instead
try:
    from importlib import metadata
    
    def discover_gsm_models():
        """Discover GSM models via entry points (modern approach)"""
        models = {}
        try:
            entry_points = metadata.entry_points(group='bmcs_matmod.gsm_models')
            for entry_point in entry_points:
                models[entry_point.name] = entry_point.load()
            return models
        except Exception as e:
            print(f"Entry point discovery failed: {e}")
            return {}

    def discover_aiida_calculations():
        """Discover AiiDA calculations via entry points (modern approach)"""
        calculations = {}
        try:
            entry_points = metadata.entry_points(group='aiida.calculations')
            for entry_point in entry_points:
                if entry_point.name.startswith('gsm.'):
                    calculations[entry_point.name] = entry_point.load()
            return calculations
        except Exception as e:
            print(f"Entry point discovery failed: {e}")
            return {}

except ImportError:
    # Fallback to pkg_resources for older Python
    import pkg_resources

    def discover_gsm_models():
        """Discover GSM models via entry points (legacy approach)"""
        models = {}
        try:
            for entry_point in pkg_resources.iter_entry_points('bmcs_matmod.gsm_models'):
                models[entry_point.name] = entry_point.load()
            return models
        except:
            return {}

    def discover_aiida_calculations():
        """Discover AiiDA calculations via entry points (legacy approach)"""
        calculations = {}
        try:
            for entry_point in pkg_resources.iter_entry_points('aiida.calculations'):
                if entry_point.name.startswith('gsm.'):
                    calculations[entry_point.name] = entry_point.load()
            return calculations
        except:
            return {}

print(f"\nDiscovered GSM models: {list(discover_gsm_models().keys())}")
print(f"Discovered AiiDA calculations: {list(discover_aiida_calculations().keys())}")

Entry Points provide multiple integration paths:
1. CLI commands (console_scripts)
2. AiiDA calculations (direct Python, no CLI needed)
3. AiiDA data types (for type discovery)
4. Custom plugin discovery

Discovered GSM models: []
Discovered AiiDA calculations: []


## 6. Best Practices for Package Architecture with Optional AiiDA

Based on our exploration, here are the recommended architectural patterns:

In [6]:
"""
RECOMMENDED ARCHITECTURE FOR BMCS_MATMOD WITH OPTIONAL AIIDA

├── bmcs_matmod/
│   ├── gsm_lagrange/
│   │   ├── core/                    # Core functionality (no AiiDA deps)
│   │   │   ├── response_data.py     # ✓ Core data class
│   │   │   ├── gsm_model.py         # ✓ Core simulation engine
│   │   │   └── response_data_viz.py # ✓ Visualization mixin
│   │   │
│   │   ├── aiida_plugin/            # AiiDA-specific code
│   │   │   ├── __init__.py          # Safe imports with fallbacks
│   │   │   ├── response_data_node.py# AiiDA DataNode (MOVE HERE)
│   │   │   ├── calculations.py      # AiiDA Calculation classes
│   │   │   ├── workflows.py         # AiiDA WorkChain classes
│   │   │   └── utils.py             # Profile management utilities
│   │   │
│   │   ├── cli/                     # Command-line interface
│   │   │   ├── cli_gsm.py          # ✓ CLI commands (can use aiida_plugin)
│   │   │   └── cli_utils.py        # CLI utilities
│   │   │
│   │   └── models/                  # Material models (no AiiDA deps)
│       
└── setup.cfg or pyproject.toml     # Entry points for discoverability
"""

# PRINCIPLE 1: Layered Dependencies
print("✓ Core modules have NO AiiDA dependencies")
print("✓ AiiDA functionality isolated in aiida_plugin/")
print("✓ CLI can optionally use AiiDA features")

# PRINCIPLE 2: Graceful Degradation
def create_response_storage(response_data, prefer_aiida=True, metadata=None):
    """Factory that chooses storage method based on availability"""
    
    if prefer_aiida and AIIDA_AVAILABLE:
        try:
            # Try AiiDA first
            from .aiida_plugin.response_data_node import create_response_data_node
            return create_response_data_node(response_data, metadata, store=False)
        except Exception as e:
            print(f"AiiDA unavailable, falling back to JSON: {e}")
    
    # Fallback to JSON storage
    return {
        "data": response_data.to_dict(),
        "metadata": metadata,
        "storage_type": "json"
    }

# PRINCIPLE 3: Profile Auto-Management
class AiiDAContext:
    """Context manager for AiiDA operations"""
    
    def __init__(self, auto_load_profile=True):
        self.auto_load_profile = auto_load_profile
        self.profile_loaded = False
        
    def __enter__(self):
        if AIIDA_AVAILABLE and self.auto_load_profile:
            try:
                aiida.load_profile()
                self.profile_loaded = True
            except:
                pass
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        pass
    
    def is_ready(self):
        return AIIDA_AVAILABLE and self.profile_loaded

# Usage example
print("\n=== Usage Examples ===")

# 1. Standalone usage (no AiiDA needed)
print("1. Standalone mode:")
rd = ResponseData(
    time=np.linspace(0, 1, 3),
    stress=np.array([0, 1, 2]),
    strain=np.array([0, 0.01, 0.02])
)
storage = create_response_storage(rd, prefer_aiida=False)
print(f"   Storage type: {storage.get('storage_type', 'aiida_node')}")

# 2. AiiDA context usage
print("2. AiiDA context mode:")
with AiiDAContext() as ctx:
    if ctx.is_ready():
        print("   ✓ AiiDA context ready")
        storage = create_response_storage(rd, prefer_aiida=True)
        print(f"   Storage: {type(storage).__name__}")
    else:
        print("   ⚠ AiiDA context not ready, using fallback")

print("\n=== Key Answers to Your Questions ===")
print("❓ Should CLI always load profile?")
print("   → NO. Use auto_load_profile=True by default, but allow override")
print()
print("❓ Where should ResponseDataNode be?") 
print("   → MOVE to aiida_plugin/ (not core/, not cli/)")
print()
print("❓ Entry points vs CLI?")
print("   → BOTH! Entry points enable direct Python integration")
print("   → CLI is just one consumer of the entry points")
print()
print("❓ Package installation without AiiDA?")
print("   → YES! Core functionality works standalone")
print("   → AiiDA is optional enhancement, not requirement")

✓ Core modules have NO AiiDA dependencies
✓ AiiDA functionality isolated in aiida_plugin/
✓ CLI can optionally use AiiDA features

=== Usage Examples ===
1. Standalone mode:
   Storage type: json
2. AiiDA context mode:
   ✓ AiiDA context ready
AiiDA unavailable, falling back to JSON: attempted relative import with no known parent package
   Storage: dict

=== Key Answers to Your Questions ===
❓ Should CLI always load profile?
   → NO. Use auto_load_profile=True by default, but allow override

❓ Where should ResponseDataNode be?
   → MOVE to aiida_plugin/ (not core/, not cli/)

❓ Entry points vs CLI?
   → BOTH! Entry points enable direct Python integration
   → CLI is just one consumer of the entry points

❓ Package installation without AiiDA?
   → YES! Core functionality works standalone
   → AiiDA is optional enhancement, not requirement
